In [18]:
from pathlib import Path
import numpy as np
import pandas as pd
from typing import List,Tuple,Callable

from sklearn.linear_model import LinearRegression
from math import log2

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import gridspec
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap,Normalize,FuncNorm
import matplotlib.patches as patches
from matplotlib.axes import Axes
from foreground import get_foreground 
from dna_features_viewer import GraphicFeature, GraphicRecord

In [19]:

def parse_diamond(matchfile:str='diamond-matches.tsv')->pd.DataFrame:
    '''
    matchfile:
        output of diamond blastp
        with '--header', 'verbose', 
        and 'tsv' suffix
    '''
    head_lines=open(matchfile,'r').readlines()[2].strip().split(': ')[1]
    diamond_aligns=pd.read_csv(matchfile,skiprows=3,names=head_lines.split(', '),delim_whitespace=True)
    return diamond_aligns

def get_identities(fasta_path:str):
    '''
    from diamond tsv
    to df of ['domain','virus','identity']
    '''
    o_dict={}
    _={}
    _['domain'],_['virus'],_['identity']=[],[],[]
    for i in Path(fasta_path).iterdir():
        if i.suffix=='.tsv':
            stem=i.stem.replace('-match','')
            diamond=parse_diamond(i.absolute())
            o_dict[stem]={i:j for i,j in zip(diamond['Subject ID'],diamond['Percentage of identical matches'])}
            for idx,s in diamond.iterrows():
                _['domain'].append(stem)
                _['virus'].append(s['Subject ID'])
                _['identity'].append(s['Percentage of identical matches'])
    identities=pd.DataFrame(_)
    # identities.columns
    return o_dict,identities

def cal_lr(x,y,refx=np.linspace(1,100,100)):
    lr=LinearRegression()
    x_=np.array(x).reshape(-1,1)
    y_=np.array(y).reshape(-1,1)
    reg=lr.fit(X=x_,y=y_)
    r2=reg.score(x_,y_)
    coef=reg.coef_[0][0]
    intercept=reg.intercept_[0]
    
    predy=reg.predict(refx.reshape(-1,1))
    sel=(predy>0) & (predy<100)
    
    reg_x=refx[sel.reshape(-1)].reshape(-1)
    reg_y=predy[sel].reshape(-1)
    
    return (
        coef,intercept,r2,
        reg_x,reg_y,
    )
    
def get_lr_cohort(identities_dict:dict,
                  ref_genome:str,
                  ref_accession:str):
    '''use `identity_dict` from `get_identities`'''
    o_dict={}
    for accession,d in identities_dict.items():
            # if accession!=ref_accession:
            x,y=[],[]
            for k in d.keys():
                if k !=ref_genome:
                    r=identities_dict[ref_accession].get(k,None)
                    if r is not None:
                        x.append(r)
                        y.append(d[k])
            if len(x)>0:
                o_dict[accession]=(x,y)
    return o_dict



def get_ref_palette(accession:str,
                    lr_metrics:dict,
                    corr_color_map:LinearSegmentedColormap,
                    fetch=lambda x: x[0],
                    transform=lambda x: 1-log2(x)):
    '''
    `lr_metrics` comes from `{k:cal_lr(v[0],v[1]) for k,v in lr_cohort.items()}`,
        where `lr_cohort` comes from get_lr_cohort
    `fetch`: fetch metric values from lr_metrics' dict
    `trasnform`: metric values mapping (usually to 0~1)
    '''
    if accession in lr_metrics:
        # ref_c=corr_color_map(lr_metrics[accession][2]/100-1e-10)
        m_val=transform(fetch(lr_metrics[accession]))
        ref_c=corr_color_map(m_val)
        return LinearSegmentedColormap.from_list(accession, [(1, 1, 1, 1), ref_c])
    else:
        return LinearSegmentedColormap.from_list(accession, [(1, 1, 1), (1, 0, 0)])
    
def get_missing_palette(domains:pd.DataFrame,ref_genome:str,
        missing_color_map:LinearSegmentedColormap)->Tuple[dict[str,Tuple[int]],int]:
    o_dict={}
    # total_genome=len(domains['genome_name'].unique())
    cano_entry=domains[domains['domain_accession']==ref_genome]['genome_name'].unique()
    uncano_entry=[i for i in domains['domain_accession'].unique() 
                    if i not in cano_entry]
    max_missing=max([len(domains[domains['domain_accession']==i]) for i in uncano_entry])
    for i in uncano_entry:
        l=len(domains[domains['domain_accession']==i])
        o_dict[i]=missing_color_map(l/max_missing)
    return o_dict,max_missing



def get_corr_color_map(lr_metrics:dict,fetch=lambda x: x[0],
                       colors = [mcolors.to_rgba(i) for i in ['teal','gold','orangered']]):
    _=[fetch(i) for i in lr_metrics.values()]
    metric_min,metric_max=min(_),max(_)
    # print(metric_min,metric_max)
    position_ref=(1-metric_min)/(metric_max-metric_min)
    if position_ref<=0:
        positions = [0, 1]
        situ='conserved'
        corr_color_map = mcolors.LinearSegmentedColormap.from_list(
            f'{situ}_color_map', [(i,j) for i,j in zip(positions,colors[:2])], N=512)
        
    elif 0<position_ref<1:
        positions = [0, position_ref, 1]
        situ='biased'
        corr_color_map = mcolors.LinearSegmentedColormap.from_list(
            f'{situ}_color_map', [(i,j) for i,j in zip(positions,colors)], N=512)
    else :
        positions = [0, 1]
        situ='diverged'
        corr_color_map = mcolors.LinearSegmentedColormap.from_list(
            f'{situ}_color_map', [(i,j) for i,j in zip(positions,colors[1:])], N=512)
    return corr_color_map,situ,position_ref,metric_min,metric_max

def hide_axes(ax:Axes,genome_start_pos:int,genome_end_pos:int,sequence_length=42000):
    '''
    hack: draw a white box over them.
    '''
    ax.add_patch(
     patches.Rectangle(
        (-100, -0.25),
        genome_start_pos,
        0.5,
        fill=True,      # remove background
        facecolor='white'
     ) ) 
    ax.add_patch(
     patches.Rectangle(
        (genome_end_pos, -0.25),
        sequence_length+100,
        0.5,
        fill=True,      # remove background
        facecolor='white'
     ) ) 

def sort_genome(genome_names:List[str],sort_rdrps:pd.DataFrame):
    o=[]
    for i in genome_names:
        if i in sort_rdrps.index:
            o.append((i,sort_rdrps.loc[i]['Percentage of identical matches']))
        else:
            o.append((i,0))
    o.sort(key=lambda x:x[1],reverse=True)
    return o
# def model_function(x:np.ndarray, a:float)->np.ndarray:
#     return 2 ** (x ** a)-1

# def cal_cur(x,y,model_function:Callable[[np.ndarray,float], np.ndarray]=model_function,p0=[2.0],refx=np.linspace(1,100,100)):
#     # lr=LinearRegression()
#     x_=np.array(x)/100
#     y_=np.array(y)/100
#     opt_paras, _ = curve_fit(model_function, x_, y_, p0=p0)
    
#     rmsd=((y_-model_function(x_, *opt_paras))**2).mean()
#     # a_optimal = optimal_parameters
#     predy = model_function(refx/100, *opt_paras)
#     sel=(predy>=0) & (predy<=1)

#     reg_x=refx[sel]
#     reg_y=predy[sel]*100

#     return (
#         opt_paras,rmsd,
#         reg_x,reg_y,
    # )

In [ ]:
def norm(x_:float):
    # return np.sqrt(x_)
    _pseudo_index=1.5#log(log(1.5,2),0.8)
    _pseudo_identity=2**(x_**_pseudo_index)-1
    return _pseudo_identity
    
def rev_norm(y_:float):
    # return y_**2
    _pseudo_index=1.5#log(log(1.5,2),0.8)
    identity=np.log2(y_+1)**(1/_pseudo_index)
    return identity

In [4]:
family_dict={
    'zika':'Flaviviridae',
    'ebola':'Filoviridae',
    'measles':'Paramyxoviridae'
}
ref_genome_dict={
    'zika':'ZIKV||AY632535',
    'ebola':'EBOV||AF086833',
    'measles':'MeV||AB016162'
}
ref_accession_dict={
    'zika':'PF00972',
    'ebola':'PF00946',
    'measles':'PF00946'
}

In [53]:


corr_color_list = [mcolors.to_rgba(i) for i in ['darkblue','teal','orangered']] #colormaps['RdYlBu_r']#['plasma_r'] ['darkgreen','darkblue','maroon']
missing_color_map = LinearSegmentedColormap.from_list('missing_color_map', [(1, 1, 1,1), mcolors.to_rgba('maroon')])


def plot_synteny(k:str,ref_genome:str,ref_accession:str,
                 corr_color_list=corr_color_list,
                 missing_color_map=missing_color_map):
    ref_palette_fetch=lambda x: x[0]

    noalign_holder=15

    domains=pd.read_csv(f'{k}-domains.csv')
    sequence_length=int(2*domains['genome_length'].max())
    ref=int(sequence_length/2)

    identities_dict=get_identities(f'{k}_fasta')[0]

    # canonical colormap
    lr_cohort=get_lr_cohort(identities_dict,ref_genome,ref_accession=ref_accession)
    lr_metrics={k:cal_lr(v[0],v[1]) for k,v in lr_cohort.items()}
    corr_color_map,situ,position_ref,metric_min,metric_max=get_corr_color_map(lr_metrics,ref_palette_fetch,corr_color_list)

    ref_palette={k:get_ref_palette(k,lr_metrics,corr_color_map,
                transform=lambda x:(x-metric_min)/(metric_max-metric_min),fetch=ref_palette_fetch) for k in lr_metrics.keys()}

    missing_palette,max_missing=get_missing_palette(domains,ref_genome,missing_color_map)
    
    plt.close('all')
    sort_rdrps=parse_diamond(f'{k}_fasta/{ref_accession}-match.tsv')
    sort_rdrps.set_index('Subject ID',inplace=True)
    genomes=domains['genome_name'].unique()



    fig,ax=plt.subplots(len(genomes)+2,1,sharex=True,figsize=(60,len(genomes)*2))
    c=2
    # fig,ax=plt.subplots(2,1,sharex=True,figsize=(30,2*2))
    gs=gridspec.GridSpec(len(genomes)+2,3)
    ax:List[Axes]

    for (i,j) in sort_genome(genomes,sort_rdrps):
        '''
        i: genome id,
        j: identity% to ref genome 
        '''
        idf=domains[domains['genome_name']==i]
        # for i,idf in domains.groupby('genome_name'):
            # sequence_length=idf.iloc[0]['genome_length']
        if (idf['domain_accession']==ref_accession).any():
            #TODO better decision of genome length & del_start
            rdrp_start=idf[idf['domain_accession']==ref_accession].iloc[0]['start']
            rdrp_hmm_start=idf[idf['domain_accession']==ref_accession].iloc[0]['hmmStart']
            del_start=ref-(rdrp_start-rdrp_hmm_start*3)
        else:
            del_start=ref-idf.iloc[0]['genome_length']/2
            # print(f'no rdrp warning:{i}') #TODO unified log system
        
        features=[]
        for _,d in idf.iterrows():
            strand=+1 if d['strand']=='SENSE' else -1
            a_=d['domain_annotation'].split(':')[0]
            '''
            a_: simplified description of domain
            #TODO annotated them in a pseudo `GraphicRecord`
            '''
            if d['domain_accession'] in ref_palette:
                identity=identities_dict[d['domain_accession']].get(i,noalign_holder)/100
                _pseudo_identity=norm(identity)
                color=ref_palette[d['domain_accession']](_pseudo_identity)
            else:
                color=missing_palette[d['domain_accession']]
            features.append(GraphicFeature(start=d['start']+del_start, end=d['end']+del_start, strand=strand, color=color,
                        label=d['domain_accession'],fontdict={'fontsize':15}))
        record = GraphicRecord(sequence_length=sequence_length, features=features)
        record.plot(ax=ax[c],figure_width=20)
        ax[c].set_facecolor("white")
        ax[c].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) # remove ticks
        ax[c].annotate(i+f'|{j}' ,(-50,-0), xycoords='data', annotation_clip=False, ha='left',fontsize=25) 
        hide_axes(ax[c],0+del_start,idf.iloc[0]['genome_length']+del_start,sequence_length=sequence_length)
        c+=1 
        
    for i in range(3):
        ax[i].set_axis_off()
    fig.tight_layout()

    annot_ax =fig.add_subplot(gs[1,:])
    virtual_track=[]
    unique_domains=domains['domain_accession'].unique()
    annot_space=np.linspace(0,sequence_length,len(unique_domains)+2)
    for i,dm in enumerate(unique_domains):
        annot=domains[domains['domain_accession']==dm].iloc[0]['domain_annotation'].split(':')[0]
        label=f'{dm}:{annot}'
        if dm in ref_palette:
            color=ref_palette[dm](1.0)
        else:
            color=missing_palette[dm]
        textcolor = get_foreground(mcolors.to_hex(color))
        virtual_track.append(GraphicFeature(start=int(annot_space[i+1]), end=int(annot_space[i+1])+1, 
                    strand=+1, color=color,label=label,
                    linewidth=0, thickness=7, box_color=color, box_linewidth=0,
                    label_link_color="white",
                    fontdict={"color":textcolor,'fontsize':15}))
    record = GraphicRecord(sequence_length=sequence_length,  features=virtual_track,feature_level_height=-1) #
    ax_, _ = record.plot(ax=annot_ax, figure_width=20,draw_line=False) 
    ax_.set_facecolor("white")
    ax_.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) # remove ticks
    # genome_start_pos = rdrp_fixed_position - [x[0] for x in sars_cov_2_features if "RdRP" in x[3]][0]
    # genome_end_pos = genome_start_pos + 29903
    # ax_.plot([genome_start_pos, genome_end_pos], [0, 0], zorder=-1000, c="k") 

    # colormap
    cbar_ax =fig.add_subplot(gs[0,0])
    cbar=fig.colorbar(cm.ScalarMappable(norm=Normalize(vmin=metric_min,vmax=metric_max), cmap=corr_color_map),cax=cbar_ax,orientation='horizontal')
    cbar_tick_locations = [metric_min,1,metric_max]#np.linspace(0,1,4)
    cbar.set_ticks(cbar_tick_locations)
    cbar.set_ticklabels([f'{i:.1f}' for i in cbar_tick_locations],fontsize=25)
    cbar.set_label('Lr `k` with ref domain',fontsize=30,loc='center')

    cbar_ax1 =fig.add_subplot(gs[0,1])
    cbar1=fig.colorbar(cm.ScalarMappable(norm=FuncNorm(functions=(norm,rev_norm),vmin=0.,vmax=1.), 
                                        cmap=ref_palette[ref_accession]),cax=cbar_ax1,orientation='horizontal')
    cbar_tick_locations = np.array([0,0.5,0.8,1])
    cbar1.set_ticks(cbar_tick_locations)
    cbar1.set_ticklabels([f'{i:.1f}' for i in cbar_tick_locations],fontsize=25)
    cbar1.set_label('Identity to RefGenome',fontsize=30,loc='center')

    cbar_ax2 =fig.add_subplot(gs[0,2])
    cbar2=fig.colorbar(cm.ScalarMappable(norm=Normalize(vmin=0.,vmax=max_missing), cmap=missing_color_map),cax=cbar_ax2,orientation='horizontal')
    cbar_tick_locations = np.array(np.linspace(0,max_missing,5))
    cbar2.set_ticks(cbar_tick_locations)
    cbar2.set_ticklabels([f'{i:.0f}' for i in cbar_tick_locations],fontsize=25)
    cbar2.set_label('Missing Gene Freq',fontsize=30,loc='center')

    # fig.set_dpi(20)
    fig.savefig(f'{k}-synteney.pdf')
    # plt.show()
    plt.close('all')

In [55]:
k='ebola'
ref_genome=ref_genome_dict[k]
ref_accession=ref_accession_dict[k]
for k in ref_genome_dict:
    plot_synteny(k,ref_genome_dict[k],ref_accession_dict[k])